In [1]:
import tensorflow as tf
from keras.applications import ResNet50, DenseNet121
from keras.layers import Conv2D, BatchNormalization
import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
from vbranch.applications.resnet import ResNet50 as vb_ResNet50
from vbranch.applications.densenet import DenseNet121 as vb_DenseNet121

In [3]:
with tf.variable_scope('keras_model'):
#     keras_model = ResNet50(weights='imagenet')
    keras_model = DenseNet121(weights='imagenet')

W0727 14:40:14.318435 139860195333888 deprecation_wrapper.py:119] From /home/gong/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 14:40:14.319051 139860195333888 deprecation_wrapper.py:119] From /home/gong/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 14:40:14.323721 139860195333888 deprecation_wrapper.py:119] From /home/gong/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 14:40:14.344909 139860195333888 deprecation_wrapper.py:119] From /home/gong/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session in

In [4]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [3]:
inputs = tf.placeholder('float32', [None, 224, 224, 3])
with tf.variable_scope('vb_model', reuse=tf.AUTO_REUSE):
#     vb_model, assign_ops = vb_ResNet50([inputs]*2, 1000, weights='imagenet', shared_frac=0.5)
    vb_model, assign_ops = vb_DenseNet121([inputs]*2, 1000, weights='imagenet', shared_frac=1)

Instructions for updating:
Colocations handled automatically by placer.
Loading weights for DenseNet121...


In [4]:
vb_model.summary()

i    Layer name                      Output shape          Num param  Inbound               
--------------------------------------------------------------------------------------------
     Input                           [None,224,224,3]                                       
--------------------------------------------------------------------------------------------
     Input                           [None,224,224,3]                                       
--------------------------------------------------------------------------------------------
0    zero_pad (ZeroPadding2D)        [None,230,230,3]      0          input                 
                                     [None,230,230,3]                                       
--------------------------------------------------------------------------------------------
1    conv1/conv (Conv2D)             [None,112,112,64] []  9408       zero_pad              
                                     [None,112,112,64] []             

In [5]:
len(vb_model.layers)

431

## Save Weights

In [5]:
imagenet_weights = {}

for layer in keras_model.layers:
    name = layer.name
    
    if isinstance(layer, Conv2D):
        # Conv
        imagenet_weights[name] = {}
        weights = layer.get_weights()
        assert len(weights) == 1
        f = weights[0]
        assert len(f.shape) == 4
#         b = weights[1]
#         assert len(b.shape) == 1
        imagenet_weights[name] = {'filter' : f} # , 'bias': b}
    elif isinstance(layer, BatchNormalization):
        # Batch norm
        imagenet_weights[name] = {}
        weights = layer.get_weights()
        gamma = weights[0]
        beta = weights[1]
        imagenet_weights[name] = {'scale' : gamma, 'beta':beta}

In [13]:
# with open('weights/densenet121.pickle', 'wb') as pickle_out:
#     pickle.dump( imagenet_weights, pickle_out)

In [3]:
# import pickle
# with open('weights/densenet121.pickle', 'rb') as pickle_in:
#     weights = pickle.load(pickle_in)

## Rearrange Weights for Densenet

In [13]:
growth_rate = 32
init_dim = 64
shared_frac = 0.5

def rearrange_4d(weight):
    fan_in = weight.shape[-2]
    fan_out = weight.shape[-1]
    
    if fan_in == init_dim:
        shared = weight[..., :int(shared_frac*fan_in), :]
        unique = weight[..., int(shared_frac*fan_in):, :]
        return shared, unique
    
    shared_1, unique_1 = rearrange_4d(weight[..., :-growth_rate, :])
    
    part2 = weight[..., -growth_rate:, :]
    shared_2 = part2[..., :int(shared_frac*growth_rate), :]
    unique_2 = part2[..., int(shared_frac*growth_rate):, :]
    
    shared = np.concatenate([shared_1, shared_2], axis=-2)
    unique = np.concatenate([unique_1, unique_2], axis=-2)
    return shared, unique

def rearrange_1d(weight):
    fan_in = weight.shape[0]
    
    if fan_in == init_dim:
        shared = weight[:int(shared_frac*fan_in)]
        unique = weight[int(shared_frac*fan_in):]
        return shared, unique
    
    shared_1, unique_1 = rearrange_1d(weight[:-growth_rate])
    
    part2 = weight[-growth_rate:]
    shared_2 = part2[:int(shared_frac*growth_rate)]
    unique_2 = part2[int(shared_frac*growth_rate):]
    
    shared = np.concatenate([shared_1, shared_2])
    unique = np.concatenate([unique_1, unique_2])
    return shared, unique

In [16]:
# weights = rearrange_4d(imagenet_weights['conv2_block2_1_conv']['filter'])
weights = rearrange_1d(imagenet_weights['conv2_block2_0_bn']['scale'])

In [17]:
weights

(array([ 1.55878544e-01,  2.50950664e-01,  8.01163092e-02,  1.57138377e-01,
         9.47337449e-02,  9.89745930e-02,  1.19960450e-01,  2.06936926e-01,
         5.82202944e-08,  1.40472814e-01,  1.14670411e-01,  1.08718365e-01,
         2.28419945e-01,  9.57213268e-02,  1.57408386e-01,  3.09053085e-08,
         1.58227503e-01, -2.18337506e-01,  1.04949726e-02,  1.17896862e-01,
         1.58270165e-01,  1.75887451e-01,  2.19314978e-01, -3.26045960e-01,
         2.83142596e-01,  1.91249892e-01,  9.96387899e-02,  1.06073029e-01,
         1.66588396e-01,  2.25092292e-01,  1.25268519e-01,  1.86098695e-01,
         2.13615388e-01,  1.23492844e-01,  2.32993722e-01,  1.18153237e-01,
         1.32339478e-01,  1.17743604e-01,  1.84976801e-01,  1.48062944e-01,
         1.20901138e-01,  1.79362550e-01,  1.35870710e-01,  1.56597003e-01,
         8.74151364e-02,  1.09723702e-01,  1.07451707e-01,  1.41890973e-01],
       dtype=float32),
 array([ 0.12102173, -0.2853337 ,  0.12858272,  0.24436137,  0.1